In [1]:
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import copy

In [2]:
# equivalent of np.random.choice with numba sipport
@jit(nopython=True)
def rand_choice_nb(k, prob):
    return np.arange(k)[np.searchsorted(np.cumsum(prob), np.random.random(), side="right")]

In [3]:
@jit(nopython=True)
def gradient_bandit(k, env, alpha1, alpha2, seed_id, steps=1000):
    '''
    params
    k: num of arms
    env: the environment represented as a tuple: (mean, cov), mean and cov are both k*1 vectors
    alpha1: step size for updating H(a)
    alpha2: step size for computing average reward R_bar
    seed_id: random seed id used to perturb the environment
    steps: the number of total time steps in a simulation
    '''
    
    # nonstaionary environment setup
    mean, cov = env
    
    # gradient bandit params initialization
    H = np.zeros(k)
    R_bar = 0
    Pi = np.exp(H) / np.sum(np.exp(H))
    
    # avg reward per timestep
    Rt = np.zeros(steps)
    
    # sum of rewards
    R_sum = 0
    
    # percentages of optimal actions at each time step
    opt_act = np.zeros(steps)
    
    # total number of optimal actions
    opt_act_sum = 0
    
    for t in range(1, steps+1):
        # select an action
        a = rand_choice_nb(k, Pi)
        
        # if an action is optimal action, increment optimal action counter
        if mean[a] == np.max(mean):
            opt_act_sum += 1
        opt_act[t-1] = opt_act_sum / t
        
        # draw a reward from environment
        R = np.random.normal(mean[a], cov[a])
        
        R_sum += R
        Rt[t-1] = R_sum / t
        
        # update H(a)
        for j in range(k):
            if j != a:
                H[j] -= alpha1 * (R - R_bar) * Pi[j]
        H[a] += alpha1 * (R - R_bar) * (1 - Pi[a])
        
        # update softmax distribution Pi
        Pi = np.exp(H) / np.sum(np.exp(H))
        
        # update average reward R_bar
        R_bar += alpha2 * (R - R_bar)
        
        # perturb the environment using random seed
        np.random.seed(seed_id)
        mean += np.random.normal(0, 1, k)
        print("gradient", mean)
            
    return Rt, opt_act

In [4]:
@jit(nopython=True)
def ucb_bandit(k, C, alpha, env, seed_id, steps=1000):
    '''
    params
    k: number of arms
    C: UCB param
    alpah: step size for updating Q (weighted average)
    env: the environment represented as a tuple: (mean, cov), mean and cov are both k*1 vectors
    seed_id: random seed id used to perturb the environment
    steps: the number of total time steps in a simulation
    '''
    
    # environment setup
    mean, cov = env
    
    # params initializations
    Q = np.zeros(k)
    N = np.zeros(k)
    
    # average reward at each time step
    Rt = np.zeros(steps)
    
    # sum of reward at each time step
    R_sum = 0
    
    # percentages of optimal actions at each time step
    Opt = np.zeros(steps)
    
    # amount of optimal actions at each time step
    Opt_sum = 0
    
    for t in range(1, steps+1):
        # Upper-confidence bound action selection
        a = Q + C * np.sqrt(np.log(t) / (N + 1e-16))
        max_actions = np.argwhere(a == np.max(a))
        At = max_actions[np.random.randint(0, len(max_actions))][0]
        
        # collect reward from the environment
        R = np.random.normal(mean[At], cov[At])
        
        # record avg reward and percentage of optimal actions
        R_sum += R
        Rt[t-1] = R_sum / t
        if mean[At] == np.max(mean):
            Opt_sum += 1
        Opt[t-1] = Opt_sum / t
        
        # update estimates and counts
        Q += alpha * (R - Q)
        N[At] += 1
        
        # perturb the environment using random seed
        np.random.seed(seed_id)
        mean += np.random.normal(0, 1, k)
        
    return Rt, Opt